In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [4]:
async def get_expiry(f_today):

    if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
        f_expiry = dt.date(2024, 1, 25)
    elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
        f_expiry = dt.date(2024, 1, 31)
    elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
        f_expiry = dt.date(2024, 2, 29)
    elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
        f_expiry = dt.date(2024, 2, 27)
    elif f_today < dt.date(2023, 9, 1):
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
    elif f_today >= dt.date(2023, 9, 1):
        if f_today.day < 24:
            days_to_wednesday = (2 - f_today.weekday()) % 7
            nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
            f_expiry = nearest_wednesday
            if nse.valid_days(
                start_date=nearest_wednesday, end_date=nearest_wednesday
            ).empty:
                f_expiry = nearest_wednesday - dt.timedelta(days=1)
        else:
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(
                start_date=nearest_thursday, end_date=nearest_thursday
            ).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry

async def get_expiry_sensex(f_today):

    days_to_thursday = (4 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [5]:
from datetime import date
from bisect import bisect_left

async def get_expiry(input_date, index_symbol='sensex'):
    expiries = dict_expiries.get(index_symbol)
    if not expiries:
        return None
        
    expiry_dates = sorted({dt.date() for dt in expiries})
    pos = bisect_left(expiry_dates, input_date)    
    return expiry_dates[pos] if pos < len(expiry_dates) else None


In [6]:
# bnf_pandas = pd.read_csv('../data/nifty_1hr_tv (2).csv')
# bnf_pandas = pd.read_csv('../data/bnf_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/midcp_select_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/sensex_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/crude_4hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/gold_4hr_tv.csv')
# bnf_1min = pd.read_csv('../data/bnf_min.csv')
bnf_1min = pd.read_csv("../data/sensex_min (1).csv")
bnf_1min.columns =  ['index', 'datetime', 'o', 'h', 'l', 'c', 'v']
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"]).dt.tz_localize(None)
# bnf_1min = bnf_1min[((bnf_1min['datetime'].dt.year == 2020) & (bnf_1min['datetime'].dt.month == 4))]
# bnf_1min = bnf_1min[
#     (bnf_1min["datetime"].dt.year >= 2017) & (bnf_1min["datetime"].dt.year <= 2024)
# ]

In [7]:
# # If Stocks Data ...
# bnf_pandas['datetime'] = pd.to_datetime(bnf_pandas['time'])
# bnf_pandas['datetime'] = bnf_pandas['datetime'].dt.tz_localize(None)
# bnf_pandas = bnf_pandas[bnf_pandas['datetime'].dt.year >= 2017]
# bnf_pandas.drop(columns=['time'], inplace=True)
# # bnf_pandas

In [8]:
# bnf = pl.DataFrame(bnf_pandas)
# bnf = bnf.with_columns([pl.col('datetime').alias('index')]).drop('datetime')
# bnf = bnf.with_columns(pl.col("index").alias("datetime"))
# # bnf

In [9]:
bnf_1min.tail()

,index,datetime,o,h,l,c,v
663206,sensex,2025-05-30 15:25:00,81425.3800,81428.3300,81408.7600,81426.2500,0
663207,sensex,2025-05-30 15:26:00,81424.4000,81435.8600,81412.7100,81418.1900,0
663208,sensex,2025-05-30 15:27:00,81420.4200,81430.8200,81407.7000,81421.0100,0
663209,sensex,2025-05-30 15:28:00,81425.7000,81425.7000,81374.9700,81412.6900,0
663210,sensex,2025-05-30 15:29:00,81412.1100,81435.4100,81384.6500,81405.8500,0


In [10]:
from expiries import dict_expiries

In [11]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [28]:
# def calculate_weekly_ranges(df):
#     # Ensure the index is a DatetimeIndex
#     df.index = pd.to_datetime(df.index)
#     # Resample to weekly data to get the high and low for each week
#     weekly_ranges = df.resample("W-WED").agg({"high": "max", "low": "min"})
#     return weekly_ranges

def calculate_weekly_levels(df):
    # Convert datetime to datetime type and set index
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.set_index('datetime')

    # Split into two parts: before and after Jan 1, 2025
    cutoff_date = pd.Timestamp('2025-01-01')

    df_before_2025 = df[df.index < cutoff_date]
    df_from_2025 = df[df.index >= cutoff_date]

    # Resample each part with appropriate week rules
    weekly_before_2025 = df_before_2025.resample('W-FRI').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last'
    })

    weekly_from_2025 = df_from_2025.resample('W-TUE').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last'
    })

    # Combine both resampled DataFrames
    weekly_df = pd.concat([weekly_before_2025, weekly_from_2025])
    weekly_df = weekly_df.sort_index()

    # Add weekly range and previous week's range
    # weekly_df['Range'] = weekly_df['high'] - weekly_df['low']
    # weekly_df['Prev_Week_Range'] = weekly_df['Range'].shift(1)
    # print(weekly_df)
    return weekly_df.reset_index()


async def find_breakouts(df, weekly_ranges):

    results = []
    multiplier_to_range = 0.4

    portfolio_value = 90_00_000
    index_leverage = 9

    index_name = "SENSEX"
    index_str_for_opt = "sensex"

    weekly_ranges["datetime"] = pd.to_datetime(weekly_ranges["datetime"])
    weekly_ranges = weekly_ranges[:-2]
    print(weekly_ranges)

    dates_to_avoid = [
        # dt.date(2017, 1, 23),
        # dt.date(2017, 3, 13),
        # dt.date(2017, 4, 10),
        # dt.date(2017, 12, 11),
        # dt.date(2019, 3, 25),
        # dt.date(2019, 4, 1),
        # dt.date(2019, 9, 9),
        # dt.date(2020, 4, 6),
        # dt.date(2020, 10, 26),
    ]

    for i in range(0, len(weekly_ranges)):
        previous_week = weekly_ranges.iloc[i - 1]
        current_week = weekly_ranges.iloc[i]
        print("Current Week : ", current_week["datetime"].date())
        if current_week["datetime"].date() in dates_to_avoid:
            print("Date Avoided")
            continue
        previous_week_high = weekly_ranges.iloc[i - 1]["high"]
        previous_week_low = weekly_ranges.iloc[i - 1]["low"]
        previous_week_close = weekly_ranges.iloc[i - 1]["close"]

        current_week_open = weekly_ranges.iloc[i]["open"]

        # Filter the 1-minute data for the current week
        # current_week_data = df[(df.index > weekly_ranges.index[i-1]) & (df.index <= current_week)]
        current_week_data = df.loc[
            (df["datetime"] >= current_week["datetime"])
            & (df["datetime"] <= (current_week["datetime"] + pd.Timedelta(days=7)))
        ]
        # print("Current Week Data:\n", current_week_data)

        weekly_range = previous_week_high - previous_week_low
        addition_range = multiplier_to_range * weekly_range

        # Check for breakouts
        high_level = current_week_data["open"].iloc[0] + addition_range
        low_level = current_week_data["open"].iloc[0] - addition_range

        # high_level = current_week_data['open'].iloc[0] + addition_range
        # low_level = current_week_data['open'].iloc[0] - addition_range
        # print(current_week)
        # print("High Level:\n", high_level, "\nLow Level:\n", low_level)

        breakout_high = current_week_data[current_week_data["high"] >= high_level]
        breakout_low = current_week_data[current_week_data["low"] <= low_level]

        if not breakout_high.empty:
            # print('Breakout High')
            breakout_high_time = breakout_high.iloc[0]["datetime"]
            # print('High Break At :', breakout_high_time)
            atm_strike = int(round(low_level / 100) * 100)
            # exit_date = current_week['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry(
                current_week["datetime"].date() + dt.timedelta(days=0)
            )
            dte = (expiry - breakout_high_time.date()).days
            contract = await get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type="PE",
            )
            pe_df = await fetch_data(
                index=index_str_for_opt,
                start_date=breakout_high_time.date(),
                end_date=expiry,
                start_time=breakout_high_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=atm_strike,
                asset_class="P",
            )
            if not isinstance(pe_df, str) and pe_df is not None:
                pe_df = pe_df.to_pandas()
                entry_price = pe_df.iloc[0]["c"]
            else:
                entry_price = float("nan")

            # Exit Scenario
            current_week_data_after_entry = current_week_data[
                (current_week_data["datetime"] > breakout_high_time)
                & (
                    current_week_data["datetime"].dt.date
                    <= expiry - dt.timedelta(days=0)
                )
            ]
            # print(current_week_data_after_entry)
            low_breach = current_week_data_after_entry[
                current_week_data_after_entry["low"] < previous_week_low
            ]
            # print(low_breach)
            if len(low_breach) != 0:
                exit_time = low_breach.iloc[0]["datetime"]
                # print(exit_time)
                remark = "SL Hit"
                # print(remark, exit_time)
                # print(pe_df)
                if not math.isnan(entry_price):
                    # print((pe_df[(pe_df['datetime'].dt.date == exit_time.date()) & (pe_df['datetime'].dt.hour >= exit_time.hour) & (pe_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    filtered_df = pe_df.loc[
                        (pe_df["datetime"].dt.date >= exit_time.date())
                        & (pe_df["datetime"].dt.hour >= exit_time.hour)
                        & (pe_df["datetime"].dt.minute >= exit_time.minute),
                        "c"]
                    exit_price = filtered_df.iloc[0] if not filtered_df.empty else float('nan')

                else:
                    exit_price = float("nan")
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(
                    expiry - dt.timedelta(days=0), dt.time(15, 30)
                )
                remark = "Expiry Closing"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = pe_df.iloc[-1]["c"]
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)
            qty = portfolio_value * index_leverage / atm_strike
            slippage = 0.01 * (entry_price + exit_price)
            pnl = qty * (entry_price - exit_price)
            final_pnl = qty * (entry_price - exit_price - slippage)
            unit = {
                "Week": previous_week["datetime"].date(),
                "Week High": previous_week_high,
                "Week Low": previous_week_low,
                "Weekly Range": weekly_range,
                "Current Week Open": current_week_open,
                "Multiplier to Range": f"{int(multiplier_to_range * 100)}%",
                "Entry Level Long": high_level,
                "Entry Level Short": low_level,
                "Break Type": "high",
                "Strike": atm_strike,
                "Expiry": expiry,
                "DTE": dte,
                "Contract": contract,
                "Option Type": contract[-2:],
                "Entry Time": breakout_high_time,
                "Entry Price": entry_price,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Remark": remark,
                "Points": entry_price - exit_price,
                "Slippage": slippage,
                "Points w cs": (entry_price - exit_price) - slippage,
                "Qty": qty,
                "PnL": pnl,
                "PnL w cs": final_pnl,
                "ROI%": (pnl * 100 / portfolio_value),
                "ROI% w cs": (final_pnl * 100 / portfolio_value),
            }
            # print(unit)
            results.append(unit)

        if not breakout_low.empty:
            # print('Breakout Low')
            breakout_low_time = breakout_low.iloc[0]["datetime"]
            # print('Low Break At :', breakout_low_time)
            atm_strike = int(round(high_level / 100) * 100)
            # exit_date = current_week['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry(
                current_week["datetime"].date() + dt.timedelta(days=0)
            )
            dte = (expiry - breakout_low_time.date()).days
            contract = await get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type="CE",
            )
            ce_df = await fetch_data(
                index=index_str_for_opt,
                start_date=breakout_low_time.date(),
                end_date=expiry,
                start_time=breakout_low_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=atm_strike,
                asset_class="C",
            )
            # print(len(ce_df))
            if not isinstance(ce_df, str) and ce_df is not None:
                ce_df = ce_df.to_pandas()
                entry_price = ce_df.iloc[0]["c"]
            else:
                entry_price = float("nan")

            # Exit Scenario
            current_week_data_after_entry = current_week_data[
                (current_week_data["datetime"] > breakout_low_time)
                & (
                    current_week_data["datetime"].dt.date
                    <= expiry - dt.timedelta(days=0)
                )
            ]
            # print(current_week_data_after_entry)
            high_breach = current_week_data_after_entry[
                current_week_data_after_entry["high"] > previous_week_high
            ]
            # print(high_breach)
            if len(high_breach) != 0:
                exit_time = high_breach.iloc[0]["datetime"]
                # print(exit_time)
                remark = "SL Hit"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    # print(exit_time.date(), exit_time.hour, exit_time.minute)
                    # print((ce_df[(ce_df['datetime'].dt.date == exit_time.date()) & (ce_df['datetime'].dt.hour >= exit_time.hour) & (ce_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    filtered_df = ce_df.loc[
                        (ce_df["datetime"].dt.date >= exit_time.date())
                        & (ce_df["datetime"].dt.hour >= exit_time.hour)
                        & (ce_df["datetime"].dt.minute >= exit_time.minute),
                        "c"]
                    exit_price = filtered_df.iloc[0] if not filtered_df.empty else float('nan') 
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(
                    expiry - dt.timedelta(days=0), dt.time(15, 30)
                )
                remark = "Expiry Closing"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = ce_df.iloc[-1]["c"]
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)

            qty = portfolio_value * index_leverage / atm_strike
            slippage = 0.01 * (entry_price + exit_price)
            pnl = qty * (entry_price - exit_price)
            final_pnl = qty * (entry_price - exit_price - slippage)
            unit = {
                "Week": previous_week["datetime"].date(),
                "Week High": previous_week_high,
                "Week Low": previous_week_low,
                "Weekly Range": weekly_range,
                "Current Week Open": current_week_open,
                "Multiplier to Range": f"{int(multiplier_to_range * 100)}%",
                "Entry Level Long": high_level,
                "Entry Level Short": low_level,
                "Break Type": "low",
                "Strike": atm_strike,
                "Expiry": expiry,
                "DTE": dte,
                "Contract": contract,
                "Option Type": contract[-2:],
                "Entry Time": breakout_low_time,
                "Entry Price": entry_price,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Remark": remark,
                "Points": entry_price - exit_price,
                "Slippage": slippage,
                "Points w cs": (entry_price - exit_price) - slippage,
                "Qty": qty,
                "PnL": pnl,
                "PnL w cs": final_pnl,
                "ROI%": (pnl * 100 / portfolio_value),
                "ROI% w cs": (final_pnl * 100 / portfolio_value),
            }
            # print(unit)
            results.append(unit)

    # print(results)
    return results

async def trade():
    df = bnf_1min
    df['datetime'] = pd.to_datetime(df['datetime'])
    # df = df.set_index('datetime')
    df = df.rename(columns={
        "o": "open",
        "h": "high",
        "l": "low",
        "c": "close"
    })
    # weekly_ranges = calculate_weekly_levels(df)
    cutoff_date = pd.Timestamp('2025-01-01')

    df_before_2025 = df[df['datetime'] < cutoff_date]
    df_from_2025 = df[df['datetime'] >= cutoff_date]
    
    df1 = resample(pl.DataFrame(df_before_2025), "7d", pd.Timedelta(days=4))
    df2 = resample(pl.DataFrame(df_from_2025), "7d", pd.Timedelta(days=6))
    weekly_ranges = pd.concat([df1.to_pandas(), df2.to_pandas()])
    weekly_ranges = weekly_ranges.reset_index()
    weekly_ranges = weekly_ranges.drop(columns=['index'])
    # weekly_ranges = weekly_ranges.to_pandas()
    # print(weekly_ranges)
    breakouts = await find_breakouts(df, weekly_ranges)
    breakouts_pandas = pd.DataFrame(breakouts)
    # breakouts_polars = pl.DataFrame(breakouts)
    return breakouts_pandas


tradebook = await trade()
# tradebook

      datetime       open       high        low      close
0   2018-03-05 33219.5200 33490.8200 33219.5200 33474.3000
1   2018-03-12 34010.9200 34068.3000 33120.3000 33253.3500
2   2018-03-19 33258.8300 33354.7700 32483.8400 32579.6700
3   2018-03-26 32561.9100 33311.0400 32515.1700 33015.6900
4   2018-04-02 33062.0900 33697.0200 32972.6300 33588.8300
5   2018-04-09 33597.1500 34313.1400 33581.3600 34185.3900
6   2018-04-16 33930.2200 34585.3900 33900.5000 34455.4400
7   2018-04-23 34476.0200 35064.8000 34343.4000 34984.4100
8   2018-04-30 35020.1700 35347.5900 34847.6100 34890.9500
9   2018-05-07 35033.9000 35596.1500 34977.8400 35511.6300
10  2018-05-14 35570.0200 35993.5300 34821.6200 34888.7300
11  2018-05-21 34900.4700 35016.1100 34302.8900 34958.9100
12  2018-05-28 35051.7900 35438.1200 34735.1100 35190.9700
13  2018-06-04 35497.7000 35628.3800 34785.0200 35411.0000
14  2018-06-11 35475.6300 35875.7500 35419.8800 35634.7100
15  2018-06-18 35712.1300 35740.9500 35249.2500 35691.83

In [29]:
tradebook.tail(40)

,Week,Week High,Week Low,Weekly Range,Current Week Open,Multiplier to Range,Entry Level Long,Entry Level Short,Break Type,Strike,Expiry,DTE,Contract,Option Type,Entry Time,Entry Price,Exit Time,Exit Price,Remark,Points,Slippage,Points w cs,Qty,PnL,PnL w cs,ROI%,ROI% w cs
346,2024-07-29,82129.4900,80868.9100,1260.5800,78588.1900,40%,79092.4220,78083.9580,high,78100,2024-08-09,4,SENSEX2480978100PE,PE,2024-08-05 09:15:00,556.2500,2024-08-05 09:16:00,543.4500,SL Hit,12.8000,10.9970,1.8030,1037.1319,13275.2881,1869.9488,0.1475,0.0208
347,2024-08-05,79984.2400,78295.8600,1688.3800,79330.1200,40%,80005.4720,78654.7680,high,78700,2024-08-16,4,SENSEX2481678700PE,PE,2024-08-12 11:16:00,159.3000,2024-08-16 15:30:00,0.1000,Expiry Closing,159.2000,1.5940,157.6060,1029.2249,163852.6048,162212.0203,1.8206,1.8024
348,2024-08-19,81236.4500,80332.6500,903.8000,81388.2600,40%,81749.7800,81026.7400,high,81000,2024-08-30,4,SENSEX24AUG81000PE,PE,2024-08-26 10:17:00,166.0000,2024-08-30 15:30:00,0.0500,Expiry Closing,165.9500,1.6605,164.2895,1000.0000,165950.0000,164289.5000,1.8439,1.8254
349,2024-08-26,82637.0300,81278.4400,1358.5900,82725.2800,40%,83268.7160,82181.8440,low,83300,2024-09-06,2,SENSEX2490683300CE,CE,2024-09-04 09:09:00,29.5500,2024-09-06 15:30:00,0.0500,Expiry Closing,29.5000,0.2960,29.2040,972.3890,28685.4742,28397.6471,0.3187,0.3155
350,2024-09-02,82725.2800,80981.9300,1743.3500,80973.7500,40%,81671.0900,80276.4100,high,80300,2024-09-13,3,SENSEX2491380300PE,PE,2024-09-10 09:09:00,47.1000,2024-09-13 15:30:00,0.0500,Expiry Closing,47.0500,0.4715,46.5785,1008.7173,47460.1494,46984.5392,0.5273,0.5221
351,2024-09-09,83116.1900,80895.0500,2221.1400,82985.3300,40%,83873.7860,82096.8740,high,82100,2024-09-20,0,SENSEX2492082100PE,PE,2024-09-20 10:36:00,3.8000,2024-09-20 15:30:00,0.2000,Expiry Closing,3.6000,0.0400,3.5600,986.6017,3551.7661,3512.3021,0.0395,0.0390
352,2024-09-16,84694.4600,82700.6300,1993.8300,84651.1500,40%,85448.6820,83853.6180,high,83900,2024-09-27,1,SENSEX24SEP83900PE,PE,2024-09-26 10:42:00,27.8000,2024-09-27 15:30:00,0.0500,Expiry Closing,27.7500,0.2785,27.4715,965.4350,26790.8224,26521.9487,0.2977,0.2947
353,2024-09-23,85978.2500,84607.3800,1370.8700,85208.7600,40%,85757.1080,84660.4120,low,85800,2024-10-04,4,SENSEX24100485800CE,CE,2024-09-30 11:25:00,149.8000,2024-10-04 15:30:00,0.0500,Expiry Closing,149.7500,1.4985,148.2515,944.0559,141372.3776,139957.7098,1.5708,1.5551
354,2024-10-07,82319.2109,80726.0625,1593.1484,81576.9297,40%,82214.1891,80939.6703,high,80900,2024-10-18,3,SENSEX24101880900PE,PE,2024-10-15 09:15:00,58.8000,2024-10-18 09:15:00,254.5000,SL Hit,-195.7000,3.1330,-198.8330,1001.2361,-195941.9036,-199078.7763,-2.1771,-2.2120
355,2024-10-07,82319.2109,80726.0625,1593.1484,81576.9297,40%,82214.1891,80939.6703,low,82200,2024-10-18,1,SENSEX24101882200CE,CE,2024-10-17 14:41:00,16.0000,2024-10-18 15:30:00,0.0500,Expiry Closing,15.9500,0.1605,15.7895,985.4015,15717.1533,15558.9964,0.1746,0.1729


In [30]:
# positive = tradebook[tradebook['Points']<0]
# positive['Points'].sum()
tradebook["ROI% w cs"].sum()
# no_value = tradebook[(tradebook['ROI%'] > 0) | (tradebook['ROI%'] < 0)]
# len(no_value)

33.43906598597562

In [31]:
# x = tradebook['Entry Time'].iloc[0].year
tradebook["Entry Time"] = pd.to_datetime(tradebook["Entry Time"])
tradebook["Trade Year"] = tradebook["Entry Time"].dt.year

In [32]:
tradebook_final = tradebook[tradebook['DTE'] <= 10]

In [33]:
stats_df8 = pd.DataFrame(
    index=range(2017, 2026),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tradebook
# Iterate over each year
for year in range(2017, 2026):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["ROI% w cs"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["ROI% w cs"] > 0).mean() * 100

    # Calculate average profit per trade

    avg_profit = year_trades[year_trades["ROI% w cs"] > 0]["ROI% w cs"].mean()
    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["ROI% w cs"] < 0]["ROI% w cs"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["ROI% w cs"].cumsum() - year_trades["ROI% w cs"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["ROI% w cs"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% w cs"] > 0][
    "ROI% w cs"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% w cs"] < 0][
    "ROI% w cs"
].mean()
overall_max_drawdown = (
    combined_df_sorted["ROI% w cs"].cumsum()
    - combined_df_sorted["ROI% w cs"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,0.0000,0,NaN,NaN,NaN,NaN,NaN
2018,0.0000,44,0.0000,NaN,NaN,NaN,NaN
2019,0.0000,51,0.0000,NaN,NaN,NaN,NaN
2020,0.0000,49,0.0000,NaN,NaN,NaN,NaN
2021,0.0000,61,0.0000,NaN,NaN,NaN,NaN
2022,0.0000,56,0.0000,NaN,NaN,NaN,NaN
2023,16.4351,52,38.4615,0.9877,-0.5532,-1.1266,14.5888
2024,18.9629,54,81.4815,0.8302,-1.7566,-9.0442,2.0967
2025,-1.9589,19,57.8947,0.5384,-2.6272,-6.5933,-0.2971
Overall,33.4391,386.0000,19.4301,0.8294,-1.5140,-11.0031,3.0391


50% MOVE VARIATION FROM CURRENT OPEN, ENTRY AT OPP LEVELS, SL AT PRV. WK H/L

In [40]:
# stats_of_trades = tradebook
# stats_of_trades['Cumulative ROI%'] = stats_of_trades.groupby('Option Type')['ROI%'].cumsum()
# stats_of_trades['Running Max ROI%'] = stats_of_trades.groupby('Option Type')['Cumulative ROI%'].cummax()
# stats_of_trades['Drawdown'] = stats_of_trades['Cumulative ROI%'] - stats_of_trades['Running Max ROI%']
# max_dd_distribution = stats_of_trades.groupby('Option Type')['Drawdown'].min().reset_index()
# max_dd_distribution.rename(columns={'Drawdown': 'Max Drawdown'}, inplace=True)
# roi_distribution = stats_of_trades.groupby('Option Type')['ROI%'].sum().reset_index()
# grouped = pd.merge(roi_distribution, max_dd_distribution, on='Option Type')
# grouped['ROI/DD Ratio'] = grouped['ROI%'] / grouped['Max Drawdown'].abs()
# grouped

In [34]:
# tradebook.to_csv('NIFTY Final TB JJMS RBOS 40pct.csv')
# tradebook2 = pd.read_csv("NIFTY Final TB JJMS RBOS 40pct.csv")
tradebook2 = tradebook
tradebook2["Expiry"] = pd.to_datetime(tradebook2["Expiry"])
tradebook2["Entry Time"] = pd.to_datetime(tradebook2["Entry Time"])
tradebook2["Exit Time"] = pd.to_datetime(tradebook2["Exit Time"])

In [35]:
def round_to_nearest_100(value):
    return 100 * round(value / 100)


async def add_hedges(df):
    # df.drop(columns=['ROI%', 'Trade Year'], inplace=True)
    hedge_pct = 2.75
    index_name = "SENSEX"
    index_str_for_opt = "sensex"

    # df['Hedge Strike'] = df.apply(lambda row: row['Strike'] + round_to_nearest_100((hedge_pct / 100) * row['Strike']) if 'C' in row['Contract'] else row['Strike'] - round_to_nearest_100((hedge_pct / 100) * row['Strike']), axis=1)
    df["Hedge Strike"] = df.apply(
        lambda row: (
            round_to_nearest_100(row["Strike"] * (1 + hedge_pct / 100))
            if row["Option Type"] == "CE"
            else round_to_nearest_100(row["Strike"] * (1 - hedge_pct / 100))
        ),
        axis=1,
    )

    df["Hedge Contract"] = df.apply(
        lambda row: get_option_contract_name2(
            index_name, row["Hedge Strike"], row["Expiry"], row["Option Type"]
        ),
        axis=1,
    )

    df["Hedge Entry Price"] = np.nan
    df["Hedge Exit Price"] = np.nan

    for i in range(0, len(df)):
        print(df.iloc[i]["Week"])
        hedge_strike = int(df.iloc[i]["Hedge Strike"])
        hedge_expiry = df.iloc[i]["Expiry"]
        hedge_opt_type = df.iloc[i]["Option Type"][-2:-1]
        hedge_entry_datetime = df.iloc[i]["Entry Time"]
        hedge_exit_datetime = df.iloc[i]["Exit Time"]

        hedge_df = await fetch_data(
            index=index_str_for_opt,
            start_date=hedge_entry_datetime.date(),
            end_date=hedge_exit_datetime.date(),
            start_time=hedge_entry_datetime.time(),
            end_time=hedge_exit_datetime.time(),
            expiry=hedge_expiry,
            strike=hedge_strike,
            asset_class=hedge_opt_type,
        )
        if not isinstance(hedge_df, str) and hedge_df is not None:
            hedge_df = hedge_df.to_pandas()
        else:
            print(hedge_df)
            df.loc[i, "Hedge Entry Price"] = 0
            df.loc[i, "Hedge Exit Price"] = 0
            continue
        # print(hedge_df)

        hedge_entry_price = hedge_df.iloc[0]["c"]
        hedge_exit_price = hedge_df.iloc[-1]["c"]

        print(hedge_entry_price, hedge_exit_price)

        df.loc[i, "Hedge Entry Price"] = hedge_entry_price
        df.loc[i, "Hedge Exit Price"] = hedge_exit_price

    return df


tb_with_hedge_tmp = await add_hedges(tradebook2)
# tb_with_hedge

2018-03-05
None
2018-03-12
None
2018-03-19
None
2018-03-26
None
2018-04-02
None
2018-04-09
None
2018-04-16
None
2018-04-23
None
2018-04-30
None
2018-05-07
None
2018-05-07
None
2018-05-14
None
2018-05-21
None
2018-05-21
None
2018-05-28
None
2018-06-04
None
2018-06-11
None
2018-06-18
None
2018-06-25
None
2018-07-02
None
2018-07-09
None
2018-07-16
None
2018-07-23
None
2018-07-30
None
2018-08-06
None
2018-08-13
None
2018-08-20
None
2018-08-27
None
2018-09-03
None
2018-09-10
None
2018-09-17
None
2018-09-24
None
2018-10-08
None
2018-10-08
None
2018-10-15
None
2018-10-22
None
2018-11-05
None
2018-11-05
None
2018-11-12
None
2018-11-19
None
2018-11-26
None
2018-12-03
None
2018-12-03
None
2018-12-17
None
2018-12-24
None
2019-01-07
None
2019-01-07
None
2019-01-14
None
2019-01-21
None
2019-01-28
None
2019-02-04
None
2019-02-11
None
2019-02-18
None
2019-02-25
None
2019-03-04
None
2019-03-18
None
2019-03-18
None
2019-04-01
None
2019-04-08
None
2019-04-15
None
2019-04-22
None
2019-04-29
None
2019-05-

In [36]:
# tb_with_hedge = pd.read_csv('JJMS TB w Hedges 1%.csv')
# tb_with_hedge

In [37]:
tb_with_hedge = tb_with_hedge_tmp

In [38]:
tb_with_hedge["Qty"] = tb_with_hedge["Qty"] * 2.75
tb_with_hedge["Hedge Points"] = (
    tb_with_hedge["Hedge Exit Price"] - tb_with_hedge["Hedge Entry Price"]
)
tb_with_hedge["PnL"] = tb_with_hedge["Points"] * tb_with_hedge["Qty"]
tb_with_hedge["PnL w cs"] = tb_with_hedge["Points w cs"] * tb_with_hedge["Qty"]
tb_with_hedge["Hedge PnL"] = tb_with_hedge["Hedge Points"] * tb_with_hedge["Qty"]
tb_with_hedge["Total PnL"] = tb_with_hedge["PnL w cs"] + tb_with_hedge["Hedge PnL"]

In [39]:
tb_with_hedge["Total ROI%"] = tb_with_hedge["Total PnL"] * 100 / 9000000
tb_with_hedge["Entry Time"] = pd.to_datetime(tb_with_hedge["Entry Time"])
tb_with_hedge["Trade Year"] = tb_with_hedge["Entry Time"].dt.year

In [40]:
# stats_of_trades = tb_with_hedge
# stats_of_trades["Cumulative ROI%"] = stats_of_trades.groupby("Option Type")[
#     "Total ROI%"
# ].cumsum()
# stats_of_trades["Running Max ROI%"] = stats_of_trades.groupby("Option Type")[
#     "Cumulative ROI%"
# ].cummax()
# stats_of_trades["Drawdown"] = (
#     stats_of_trades["Cumulative ROI%"] - stats_of_trades["Running Max ROI%"]
# )
# max_dd_distribution = (
#     stats_of_trades.groupby("Option Type")["Drawdown"].min().reset_index()
# )
# max_dd_distribution.rename(columns={"Drawdown": "Max Drawdown"}, inplace=True)
# roi_distribution = (
#     stats_of_trades.groupby("Option Type")["Total ROI%"].sum().reset_index()
# )
# grouped = pd.merge(roi_distribution, max_dd_distribution, on="Option Type")
# grouped["ROI/DD Ratio"] = grouped["Total ROI%"] / grouped["Max Drawdown"].abs()
# grouped

In [41]:
stats_df8 = pd.DataFrame(
    index=range(2017, 2026),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tb_with_hedge
# Iterate over each year
for year in range(2017, 2026):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["Total ROI%"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["Total ROI%"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["Total ROI%"] > 0]["Total ROI%"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["Total ROI%"] < 0]["Total ROI%"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["Total ROI%"].cumsum() - year_trades["Total ROI%"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["Total ROI%"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["Total ROI%"] > 0][
    "Total ROI%"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["Total ROI%"] < 0][
    "Total ROI%"
].mean()
overall_max_drawdown = (
    combined_df_sorted["Total ROI%"].cumsum()
    - combined_df_sorted["Total ROI%"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,0.0000,0,NaN,NaN,NaN,NaN,NaN
2018,0.0000,44,0.0000,NaN,NaN,NaN,NaN
2019,0.0000,51,0.0000,NaN,NaN,NaN,NaN
2020,0.0000,49,0.0000,NaN,NaN,NaN,NaN
2021,0.0000,61,0.0000,NaN,NaN,NaN,NaN
2022,0.0000,56,0.0000,NaN,NaN,NaN,NaN
2023,39.5100,52,38.4615,2.4424,-1.5562,-3.1321,12.6146
2024,35.5184,54,81.4815,1.9374,-4.9729,-25.3953,1.3986
2025,-2.8418,19,57.8947,1.2922,-5.6852,-13.5114,-0.2103
Overall,72.1866,386.0000,19.4301,1.9774,-4.0064,-28.2371,2.5564


In [42]:
tb_with_hedge.to_csv('sensex_rbw.csv')

In [27]:
a = tb_with_hedge["PnL"].sum()
b = tb_with_hedge["Hedge PnL"].sum()
c = tb_with_hedge["Total PnL"].sum()
d = tb_with_hedge["Total ROI%"].sum()
print(a, b, c, d)

13098306.569737973 757314.0041415319 8639849.608152946 95.99832897947721


In [28]:
# tb_with_hedge.to_csv('JJMS TB w Hedges 1%.csv')

In [29]:
# # tb_with_hedge['DD%'] = (tb_with_hedge['Total ROI%'].cumsum() - tb_with_hedge['Total ROI%'].cumsum().cummax())
# tb_with_hedge.drop(columns=['Cumulative ROI%', 'Running Max ROI%', 'Drawdown'], inplace=True)
# tb_with_hedge

In [30]:
# tb_with_hedge["ROI%"] = tb_with_hedge["PnL"] * 100 / 1000000

In [31]:
tb_with_hedge.tail(5)

,Week,Week High,Week Low,Weekly Range,Current Week Open,Multiplier to Range,Entry Level Long,Entry Level Short,Break Type,Strike,Expiry,DTE,Contract,Option Type,Entry Time,Entry Price,Exit Time,Exit Price,Remark,Points,Slippage,Points w cs,Qty,PnL,PnL w cs,ROI%,ROI% w cs,Trade Year,Hedge Strike,Hedge Contract,Hedge Entry Price,Hedge Exit Price,Hedge Points,Hedge PnL,Total PnL,Total ROI%
372,2025-04-18,80254.5500,78776.0600,1478.4900,79830.1500,40%,80421.5460,79238.7540,low,80400,2025-05-06,11,SENSEX2550680400CE,CE,2025-04-25 10:00:00,400.3000,2025-04-28 12:28:00,705.4000,SL Hit,-305.1000,11.0570,-316.1570,2770.5224,-845286.3806,-875920.0466,-3.4153,-3.5391,2025,82600,SENSEX2550682600CE,75.8500,108.6000,32.7500,90734.6082,-785185.4384,-8.7243
373,2025-04-25,80661.3100,78605.8100,2055.5000,80300.1900,40%,81122.3900,79477.9900,high,79500,2025-05-13,11,SENSEX2551379500PE,PE,2025-05-02 09:57:00,379.9000,2025-05-13 15:30:00,0.1500,Expiry Closing,379.7500,3.8005,375.9495,2801.8868,1064016.5094,1053367.9387,4.2991,4.2560,2025,77300,SENSEX2551377300PE,107.7000,0.0500,-107.6500,-301623.1132,751744.8255,8.3527
374,2025-05-02,81177.9300,79937.4800,1240.4500,78968.3400,40%,79464.5200,78472.1600,high,78500,2025-05-20,11,SENSEX2552078500PE,PE,2025-05-09 09:15:00,737.0000,2025-05-09 09:16:00,722.7000,SL Hit,14.3000,14.5970,-0.2970,2837.5796,40577.3885,-842.7611,0.1639,-0.0034,2025,76300,SENSEX2552076300PE,0.0000,0.0000,0.0000,0.0000,-842.7611,-0.0094
375,2025-05-09,82718.1400,78968.3400,3749.8000,82392.6300,40%,83892.5500,80892.7100,low,83900,2025-05-27,5,SENSEX25MAY83900CE,CE,2025-05-22 09:23:00,40.5000,2025-05-27 15:30:00,0.0500,Expiry Closing,40.4500,0.4055,40.0445,2654.9464,107392.5805,106315.9997,0.4339,0.4296,2025,86200,SENSEX25MAY86200CE,7.2000,0.0500,-7.1500,-18982.8665,87333.1332,0.9704
376,2025-05-16,82514.8100,80489.9200,2024.8900,80897.0000,40%,81706.9560,80087.0440,high,80100,2025-06-24,32,SENSEX25JUN80100PE,PE,2025-05-23 10:47:00,699.9500,2025-06-24 15:30:00,644.8000,Expiry Closing,55.1500,13.4475,41.7025,2780.8989,153366.5730,115970.4354,0.6197,0.4686,2025,77900,SENSEX25JUN77900PE,0.0000,0.0000,0.0000,0.0000,115970.4354,1.2886


In [120]:
stats_df9 = pd.DataFrame(
    index=range(2017, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tb_with_hedge
# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["Total ROI%"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["Total ROI%"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["Total ROI%"] > 0]["Total ROI%"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["Total ROI%"] < 0]["Total ROI%"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["Total ROI%"].cumsum() - year_trades["Total ROI%"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df9.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df9["Total ROI"].sum()
overall_total_trades = stats_df9["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["ROI% w cs"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["Total ROI%"] > 0][
    "Total ROI%"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["Total ROI%"] < 0][
    "Total ROI%"
].mean()
overall_max_drawdown = (
    combined_df_sorted["Total ROI%"].cumsum()
    - combined_df_sorted["Total ROI%"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df9.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df9

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,0.0000,0,NaN,NaN,NaN,NaN,NaN
2018,0.0000,0,NaN,NaN,NaN,NaN,NaN
2019,0.0000,0,NaN,NaN,NaN,NaN,NaN
2020,0.0000,0,NaN,NaN,NaN,NaN,NaN
2021,0.0000,0,NaN,NaN,NaN,NaN,NaN
2022,0.0000,0,NaN,NaN,NaN,NaN,NaN
2023,18.8665,17,70.5882,2.0148,-1.0623,-2.2466,8.3978
2024,57.9922,53,77.3585,2.4083,-5.8214,-23.4470,2.4733
Overall,76.8587,70.0000,77.1429,2.3192,-3.8384,-23.4470,3.2780


In [121]:
# tradebook["Cumulative ROI%"] = tradebook["ROI% w cs"].cumsum()
# tradebook["Running Max ROI%"] = tradebook["Cumulative ROI%"].cummax()
# tradebook["Drawdown"] = tradebook["Cumulative ROI%"] - tradebook["Running Max ROI%"]
# tradebook

In [122]:
tradebook.to_csv("RBW_SENSEX.csv", index=False)

In [118]:
async def find_breakouts_monthly(df, monthly_ranges):

    results = []
    multiplier_to_range = 0.5

    portfolio_value = 10_00_000
    index_leverage = 10

    index_name = "NIFTY"
    index_str_for_opt = "nifty"

    monthly_ranges["datetime"] = pd.to_datetime(monthly_ranges["datetime"])
    monthly_ranges = monthly_ranges[:-1]
    # print(weekly_ranges)

    dates_to_avoid = [
        # dt.date(2017, 1, 23),
        # dt.date(2017, 3, 13),
        # dt.date(2017, 4, 10),
        # dt.date(2017, 12, 11),
        # dt.date(2019, 3, 25),
        # dt.date(2019, 4, 1),
        # dt.date(2019, 9, 9),
        # dt.date(2020, 4, 6),
        # dt.date(2020, 10, 26),
    ]
    
    for i in range(1, len(monthly_ranges)):
        previous_mmonth = monthly_ranges.iloc[i - 1]
        current_mmonth = monthly_ranges.iloc[i]
        print("Current mmonth : ", current_mmonth["datetime"].date())
        if current_mmonth["datetime"].date() in dates_to_avoid:
            print("Date Avoided")
            continue
        previous_mmonth_high = monthly_ranges.iloc[i - 1]["high"]
        previous_mmonth_low = monthly_ranges.iloc[i - 1]["low"]
        previous_mmonth_close = monthly_ranges.iloc[i - 1]["close"]

        current_mmonth_open = monthly_ranges.iloc[i]["open"]

        # Filter the 1-minute data for the current mmonth
        # current_mmonth_data = df[(df.index > mmonthly_ranges.index[i-1]) & (df.index <= current_mmonth)]
        current_mmonth_data = df.loc[
            (df["datetime"] >= current_mmonth["datetime"])
            & (df["datetime"] <= (current_mmonth["datetime"] + pd.Timedelta(days=7)))
        ]
        # print("Current mmonth Data:\n", current_mmonth_data)

        monthly_range = previous_mmonth_high - previous_mmonth_low
        addition_range = multiplier_to_range * monthly_range

        # Check for breakouts
        high_level = current_mmonth_data["open"].iloc[0] + addition_range
        low_level = current_mmonth_data["open"].iloc[0] - addition_range

        # high_level = current_week_data['open'].iloc[0] + addition_range
        # low_level = current_week_data['open'].iloc[0] - addition_range
        # print(current_week)
        # print("High Level:\n", high_level, "\nLow Level:\n", low_level)

        breakout_high = current_mmonth_data[currentmmonthk_data["high"] >= high_level]
        breakout_low = current_mmonth_data[current_mmonth_data["low"] <= low_level]

        if not breakout_high.empty:
            # print('Breakout High')
            breakout_high_time = breakout_high.iloc[0]["datetime"]
            # print('High Break At :', breakout_high_time)
            atm_strike = int(round(low_level / 100) * 100)
            # exit_date = currentmmonthk['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry_sensex(
                current_mmonth["datetime"].date() + dt.timedelta(days=5)
            )
            dte = (expiry - breakout_high_time.date()).days
            contract = await get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type="PE",
            )
            pe_df = await fetch_data(
                index=index_str_for_opt,
                start_date=breakout_high_time.date(),
                end_date=expiry,
                start_time=breakout_high_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=atm_strike,
                asset_class="P",
            )
            if not isinstance(pe_df, str) and pe_df is not None:
                pe_df = pe_df.to_pandas()
                entry_price = pe_df.iloc[0]["c"]
            else:
                entry_price = float("nan")

            # Exit Scenario
            current_mmonth_data_after_entry = current_mmonth_data[
                (current_mmonth_data["datetime"] > breakout_high_time)
                & (
                    current_mmonth_data["datetime"].dt.date
                    <= expiry - dt.timedelta(days=0)
                )
            ]
            # print(current_mmonth_data_after_entry)
            low_breach = current_mmonth_data_after_entry[
                current_mmonth_data_after_entry["low"] < previous_mmonth_low
            ]
            # print(low_breach)
            if len(low_breach) != 0:
                exit_time = low_breach.iloc[0]["datetime"]
                # print(exit_time)
                remark = "SL Hit"
                # print(remark, exit_time)
                # print(pe_df)
                if not math.isnan(entry_price):
                    # print((pe_df[(pe_df['datetime'].dt.date == exit_time.date()) & (pe_df['datetime'].dt.hour >= exit_time.hour) & (pe_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    filtered_df = pe_df.loc[
                        (pe_df["datetime"].dt.date >= exit_time.date())
                        & (pe_df["datetime"].dt.hour >= exit_time.hour)
                        & (pe_df["datetime"].dt.minute >= exit_time.minute),
                        "c"]
                    exit_price = filtered_df.iloc[0] if not filtered_df.empty else float('nan')
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(
                    expiry - dt.timedelta(days=0), dt.time(15, 30)
                )
                remark = "Friday Closing"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = pe_df.iloc[-1]["c"]
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)
            qty = portfolio_value * index_leverage / atm_strike
            slippage = 0.01 * (entry_price + exit_price)
            pnl = qty * (entry_price - exit_price)
            final_pnl = qty * (entry_price - exit_price - slippage)
            unit = {
                "mmonth": previous_mmonth["datetime"].date(),
                "mmonth High": previous_mmonth_high,
                "mmonth Low": previous_mmonth_low,
                "Monthly Range": monthly_range,
                "Current mmonth Open": current_mmonth_open,
                "Multiplier to Range": f"{int(multiplier_to_range * 100)}%",
                "Entry Level Long": high_level,
                "Entry Level Short": low_level,
                "Break Type": "high",
                "Strike": atm_strike,
                "Expiry": expiry,
                "DTE": dte,
                "Contract": contract,
                "Option Type": contract[-2:],
                "Entry Time": breakout_high_time,
                "Entry Price": entry_price,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Remark": remark,
                "Points": entry_price - exit_price,
                "Slippage": slippage,
                "Points w cs": (entry_price - exit_price) - slippage,
                "Qty": qty,
                "PnL": pnl,
                "PnL w cs": final_pnl,
                "ROI%": (pnl * 100 / portfolio_value),
                "ROI% w cs": (final_pnl * 100 / portfolio_value),
            }
            # print(unit)
            results.append(unit)

        if not breakout_low.empty:
            # print('Breakout Low')
            breakout_low_time = breakout_low.iloc[0]["datetime"]
            # print('Low Break At :', breakout_low_time)
            atm_strike = int(round(high_level / 100) * 100)
            # exit_date = current_mmonth['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry_sensex(
                current_mmonth["datetime"].date() + dt.timedelta(days=5)
            )
            dte = (expiry - breakout_low_time.date()).days
            contract = await get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type="CE",
            )
            ce_df = await fetch_data(
                index=index_str_for_opt,
                start_date=breakout_low_time.date(),
                end_date=expiry,
                start_time=breakout_low_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=atm_strike,
                asset_class="C",
            )
            # print(len(ce_df))
            if not isinstance(ce_df, str) and ce_df is not None:
                ce_df = ce_df.to_pandas()
                entry_price = ce_df.iloc[0]["c"]
            else:
                entry_price = float("nan")

            # Exit Scenario
            current_mmonth_data_after_entry = current_mmonth_data[
                (current_mmonth_data["datetime"] > breakout_low_time)
                & (
                    current_mmonth_data["datetime"].dt.date
                    <= expiry - dt.timedelta(days=0)
                )
            ]
            # print(current_mmonth_data_after_entry)
            high_breach = current_mmonth_data_after_entry[
                current_mmonth_data_after_entry["high"] > previous_mmonth_high
            ]
            # print(high_breach)
            if len(high_breach) != 0:
                exit_time = high_breach.iloc[0]["datetime"]
                # print(exit_time)
                remark = "SL Hit"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    # print(exit_time.date(), exit_time.hour, exit_time.minute)
                    # print((ce_df[(ce_df['datetime'].dt.date == exit_time.date()) & (ce_df['datetime'].dt.hour >= exit_time.hour) & (ce_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    filtered_df = ce_df.loc[
                        (ce_df["datetime"].dt.date >= exit_time.date())
                        & (ce_df["datetime"].dt.hour >= exit_time.hour)
                        & (ce_df["datetime"].dt.minute >= exit_time.minute),
                        "c"]
                    exit_price = filtered_df.iloc[0] if not filtered_df.empty else float('nan')
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(
                    expiry - dt.timedelta(days=0), dt.time(15, 30)
                )
                remark = "Expiry Closing"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = ce_df.iloc[-1]["c"]
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)

            qty = portfolio_value * index_leverage / atm_strike
            slippage = 0.01 * (entry_price + exit_price)
            pnl = qty * (entry_price - exit_price)
            final_pnl = qty * (entry_price - exit_price - slippage)
            unit = {
                "month": previous_month["datetime"].date(),
                "month High": previous_month_high,
                "month Low": previous_month_low,
                "Monthly Range": monthly_range,
                "Current month Open": current_month_open,
                "Multiplier to Range": f"{int(multiplier_to_range * 100)}%",
                "Entry Level Long": high_level,
                "Entry Level Short": low_level,
                "Break Type": "low",
                "Strike": atm_strike,
                "Expiry": expiry,
                "DTE": dte,
                "Contract": contract,
                "Option Type": contract[-2:],
                "Entry Time": breakout_low_time,
                "Entry Price": entry_price,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Remark": remark,
                "Points": entry_price - exit_price,
                "Slippage": slippage,
                "Points w cs": (entry_price - exit_price) - slippage,
                "Qty": qty,
                "PnL": pnl,
                "PnL w cs": final_pnl,
                "ROI%": (pnl * 100 / portfolio_value),
                "ROI% w cs": (final_pnl * 100 / portfolio_value),
            }
            # print(unit)
            results.append(unit)

    # print(results)
    return results

In [124]:
async def trade():
    bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"]).dt.tz_localize(None)
    print(bnf_1min.head())
    df = bnf_1min
    # weekly_ranges = calculate_weekly_ranges(df)
    monthly_ranges = resample(pl.DataFrame(df), "1mo", pd.Timedelta(days=0))
    monthly_ranges = monthly_ranges.to_pandas()
    print(monthly_ranges)
    # breakouts = await find_breakouts_monthly(df, monthly_ranges)
    # breakouts_pandas = pd.DataFrame(breakouts)
    # # breakouts_polars = pl.DataFrame(breakouts)
    # return breakouts_pandas


tradebook = await trade()
# tradebook

             datetime       open       high        low      close
0 2018-03-08 09:15:00 33219.5200 33292.2200 33219.5200 33286.7100
1 2018-03-08 09:16:00 33283.4200 33306.6300 33275.7700 33306.0200
2 2018-03-08 09:17:00 33279.8700 33279.8700 33256.5300 33258.1200
3 2018-03-08 09:18:00 33264.2600 33288.4800 33264.2600 33282.4700
4 2018-03-08 09:19:00 33271.1500 33271.1500 33264.9200 33266.2300
     datetime       open       high        low      close
0  2018-03-01 33219.5200 34068.3000 32483.8400 33015.6900
1  2018-04-01 33062.0900 35212.9500 32972.6300 35173.0300
2  2018-05-01 35347.5900 35993.5300 34302.8900 35287.4900
3  2018-06-01 35323.0300 35875.7500 34785.0200 35402.9100
4  2018-07-01 35536.3800 37644.1700 35106.5700 37599.4600
5  2018-08-01 37622.6300 38975.2900 37128.9900 38613.8000
6  2018-09-01 38918.4500 38918.4500 35987.2300 36222.3100
7  2018-10-01 36274.4900 36615.2300 33292.3500 34370.3900
8  2018-11-01 34650.6300 36389.2200 34303.8700 36240.1800
9  2018-12-01 36406.4100